In [1]:
%pip install fuzzywuzzy
%pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import botocore
import boto3
import json
import time
import requests
import datetime

import pandas as pd
import numpy as np

from sagemaker.session import Session
from difflib import SequenceMatcher
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz

## Accounting term matching
**Check to see if we report totals (e.g. Total Liability & Shareholder Equity) or sub-totals (e.g. Total Financial Instruments), these figure are not needed for construction of the unstructured database (avoid classification issue)**

In [3]:
# numpy exception for handling invalid log10 RunTime error (we opt to not show)
# switch 'ignore' to 'warn', if you would like to flag the RunTime error 
np.seterr(invalid = 'ignore') 

def multiple_check(x1:float, x2:float):
    """
    Determine whether the two values are the same number scaled by 10
    """
    # prevent zero division error since x1 is the denominator and log10 zero division error
    if (x1 == 0) or (x2 == 0): return (x1, False)
    else:
        # if our backward sum is a multiple of 10, we return True 
        # (e.g. Total Assets (x1) 745.2322 vs Backward Sum (x2) 7452322)
        check1 = np.log10(x2 / x1).is_integer()

        # if our backward sum is a substring of a line item, with a difference of one in length, we return True 
        # (e.g. Total Assets (x1) 174182935 vs Backward Sum (x2) 74182935)
        check2 = (str(x2) in str(x1) ) & (len(str(x2)) == len(str(x1)) - 1)

        if check1 or check2:
            return (x2, True)
        else: 
            return (x1, False)

def epsilon_error(x1:float, x2:float, tol:float=0.01):
    """
    Determine whether the two values are within a similar epsilon bound. We default our error tolerance
    to 0.01, implying that if two numbers are within a specified toleracnce (default 1% ) of one another 
    """
    if (x1 == 0) or (x2 == 0): return False
    else:
        # first we convert the numeric quantities into strings
        current = str(x1)
        lookback = str(x2)

        # we only want to check against the relative difference if one element in the number is read wrong
        if len(current) == len(lookback):
            
            # we iterate linearly through each string and check to see the positional match 
            # if we catch a mismatch we flag it with a 1, othewise skip with 0
            changes = [0 if (current[i] == lookback[i]) else 1 for i in range(len(current))]

            # check set differences produce a set with exactly 1 in length
            if sum(changes) == 1:

                diff = abs(x1 - x2)      # compute numeric differences

                # check to see whether an accounting condition was met wihtin a boundary condition
                if abs(diff / x1) <= tol:
                    return True

        return False

In [4]:
def totals_check(df:pd.DataFrame) -> tuple:
    """
    Checks to see if a line row meets the conditon of a total, if true we remove these rows as we make 
    have checked the terms before have meet our conditions (these include major and minor totals)
    ------------------------------------------------------------------------------------
    :param: (type pandas.DataFrame)
        A DataFrame that represents the Asset or Liability & Equity portion of the balance sheet
        
    :return: (type tuple)
        Return a cleaned DataFrame that strips the rows that represent totals
    """
    m, n = df.shape                  # unpack the shape of dataframe
    data_col = df.columns[1]         # the values column for balance sheet
    
    total_flag = 2       # default 2 (no measure found), 1 (sum is correct), 0 (sum is not correct)
    total_amt = np.nan
    
    # iterate through each of the line items
    for i in range(m):
        
        # check the value of line items at a given index (forward index)
        item1 = df.loc[i].values[1]
        name = df.loc[i].values[0]
        
        # ------------------------------------------------------------------
        # Perform regex search to determine "special" total rows
        # ------------------------------------------------------------------
        a_check = re.search('total assets', name, flags=re.I)
        le_check = re.search('(?=.*(liability|liabilities))(?=.*(equity|deficit|capital))', 
                             name, flags=re.I)
        # ------------------------------------------------------------------
        
        # if we find either total measure we re-write indicators
        if a_check is not None or le_check is not None:
            total_flag = 0; total_amt = item1;
        
        # compute backward sum (lookback index) 
        for j in range(i):
            
            # check whether dataframe empty (if so we skip to avoid fitting errors)
            # NOTE: Index position (i-1)   = line above current line
            #                      (i-j-1) = trailing look up line 'j' lines above the line above current line
            lookback = df.loc[i-j-1:i-1][data_col]
            
            # we check whether the lookback period is empty (if so we most likely deleted the row)
            if not lookback.empty:
                # backward sum for line items (index minus j-periods before)
                item2 = lookback.sum()

                # if we achieve this then we strip totals and break, no need to continue backward sum
                check1 = item1 == item2
                val, check2 = multiple_check(item1, item2)
                check3 = epsilon_error(item1, item2, tol=0.01)
                
                if check1 or check2 or check3:
                    df = df.drop(index=i)
                    
                    # if we drop the "Total" line-item then we re-assign flag to 1
                    if a_check is not None or le_check is not None:
                        total_flag = 1
                        total_amt = val
                    
                    # Error Handling for row deletions (uncomment for when not in use)
                    print('\tWe dropped row {}, {}, with lookback window of {}.'.format(i, name, j+1))
                    print('\t\tOur row is valued at {}, our lookback sum is {}'.format(item1, item2))
                    
                    # we break from inner loop to avoid key error flag 
                    break     
                
    return (df, total_flag, total_amt)

## Merging PDFs and PNGs 
**Functions to combine PDFs and PNGs where rows may be omitted**

In [5]:
def special_merge(df1:pd.DataFrame, df2:pd.DataFrame, col:str) -> pd.DataFrame:
    """
    Special type of merge for dataframes, combining all unique row items for a specified column. 
    This is designed to combine PDF and PNG balance sheets that differ in one or more rows.
    ------------------------------------------------------------------------------------
    :param: (type pandas.DataFrame)
        A DataFrame that represents either the PDF or PNG retreived from Balance Sheet
    :param: (type pandas.DataFrame)
        A DataFrame that represents either the PDF or PNG retreived from Balance Sheet
    :param: (type str)
        A shared column name that exists in both pandas.DataFrames (i.e. df1, df2)
        
    :return: (type pandas.DataFrame)
        Return a cleaned DataFrame that merges any row that was omitted, changed or missing
    """
    arr1 = df1[col].values
    arr2 = df2[col].values
    concat_pdf = []
    
    # find the sequences that match between either lineitems
    sm = SequenceMatcher(a=arr1, b=arr2)
    
    # The SequenceMathcer returns a 5-tupled for each correspond "obj"
    # 'replace'     a[i1:i2] should be replaced by b[j1:j2].
    # 'delete'      a[i1:i2] should be deleted. Note that j1 == j2 in this case.
    # 'insert'      b[j1:j2] should be inserted at a[i1:i1]. Note that i1 == i2 in this case.
    # 'equal'       a[i1:i2] == b[j1:j2] (the sub-sequences are equal)
    for (obj, i1, i2, j1, j2) in sm.get_opcodes():
        
        # implies that we want to "replace" the left side elment with the corresponding
        # right side element at the same index position (we perseve both)
        if obj == 'replace':
            
            # check the value of a fuzzy match, only insert both rows if they vastly different
            left_names = arr1[i1:i2]
            right_names = arr2[j1:j2]
            
            # iterate through each of the checks
            for it, (left, right) in enumerate(zip(left_names, right_names)):
                
                # compute the fuzz match between string (how close are these values)
                score = fuzz.partial_ratio(left.lower(), right.lower())

                # if not close in match then we append both values
                if score < 90:
                    concat_pdf.append(df1.iloc[i1:i1+it+1])
                    concat_pdf.append(df2.iloc[j1:j1+it+1])
                else:
                    concat_pdf.append(df1.iloc[i1:i1+it+1])
        
        # implies that we want to "delete" the left side element (we preserve this side)
        elif obj == 'delete':
            concat_pdf.append(df1.iloc[i1:i2])
        
        # implied that we want to "insert" the right side element (we preserve this side)
        elif obj == 'insert':
            concat_pdf.append(df2.iloc[j1:j2])
            
        elif obj == 'equal':
            concat_pdf.append(df1.iloc[i1:i2])
    
    # return concated pandas.DataFrame and reset index, removing old index
    return pd.concat(concat_pdf).reset_index(drop=True)
    

## Unstructured Database construction
**We develop our unstructured database from each of the non-total rows (concating the line items)**

In [6]:
def unstructured_data(df, filing_d, fiscal_y, cik, cik2name:dict) -> pd.DataFrame:
    """
    Forms unstructured row for larger database to be stored in s3 bucket
    ------------------------------------------------------------------------------------------
    Input:
        :param: df (type pandas.DataFrame)
            The balance sheet for a particular  
        :paran: filing_d (type str)
            The filing date for release of X-17A-5 filings for a broker dealer e.g. 2013-03-21
        :paran: fiscal_y (type str)
            The fiscal year for the balance sheet to cover e.g. 2012 (usually 1-year prior to filing date)
        :paran: cik (type str)
            The CIK number for a broker dealer e.g. 887767
        :paran: cik2name (type dict)
            A dictionary that maps CIK to Broker Deale names 
    Output:  
        :return: (type pandas.DataFrame)
             Return a transposed dataframe with additional columns corresponding to filing data
    """
    
    # intialize the first column (line items)
    first_column = df.columns[0]
    
    # clean dataframe should be of size greater than 1
    if len(df.columns) > 1:
        
        # transpose split balance sheet figure (our line items are now columns for DataFrame)
        # we first groupby the first column (this become index) and sum to group congruent names
        row = df.groupby(first_column).sum(min_count=1).T
        
        # creating additional columns in row
        row['CIK'] = cik                                  # CIK number for firm 
        row['Filing Date'] = filing_d                     # Filing Date for firm filing
        row['Filing Year'] = fiscal_y                     # Year for balance sheet filing
        row['Name'] = cik2name['broker-dealers'][cik]     # returns the name of associated with the CIK
        
        return row
    
    else:
        print('{}-{}.csv - encountered issue reading PDF'.format(cik, filing_d))
        return None
    

In [7]:
def extra_cols(csv_name:str):
    """
    Check to see whether their exists the presence of a total term from the line items
    ------------------------------------------------------------------------------------------
    Input:
        :param: csv_name (type str)
            The file directory on the s3 where data is stored (e.g. )
    Output:  
        :return: (type tuple)
            Returns a tuple for corresponding (file_name, filing_date, fiscal_year, cik)
    """
    
    file_name = csv_name.split('/')[-1]        # e.g. '1224385-2005-03-01.csv'
    csv_strip = file_name[:-4]                 # ignore last four elements from the back (i.e. .csv)

    # construct a string measure of important data measures 
    data_split = csv_strip.split('-')              
    filing_date = '-'.join(data_split[1:])         # join YYYY-mm-dd component for filing date
    fiscal_year = int(data_split[1]) - 1           # fiscal year are generally the previous year of filing
    cik = data_split[0]                            # extract the CIK number  
    
    return (file_name, filing_date, fiscal_year, cik)   

In [8]:
def reorder_columns(df:pd.DataFrame, col_preserve:list) -> pd.DataFrame:
    """
    Re-order the completed DataFrame by ordering the CIK, Name, Filing Data and Filing Year 
    ------------------------------------------------------------------------------------------
    Input:
        :param: df (type pandas.DataFrame)
            The unstructured database for balance sheet figures
    Output:  
        :return: (type pandas.DataFrame)
            Return a dataframe with dimensions less than or equal to input dataframe (MxN) -> (MxK), 
            where K <= N
    """
    # re-order the CIK and Year columns to appear as the first two columns
    remap = df.columns[~np.isin(df.columns, col_preserve)]                             
    df = df[np.insert(remap,                                       # pass all other columns, not in preserve list
                      np.zeros(len(col_preserve), dtype=int),      # map the location to the first index (i.e. 0)
                      col_preserve)]                               # insert columns we wished to preserve 

    filterNaN = df.isnull().all()                      # find if any column is all NaN 
    cleanCols = filterNaN[filterNaN == False].index    # select columns with at least one value

    # clean dataframe for unstructured asset terms
    return df[cleanCols]

## Final Main Execution

In [10]:
if __name__ == "__main__":
    
    # initiate s3 bucket and corresponding data folder
    bucket = "ran-s3-systemic-risk"
    
    pdf_asset_folder = "Output/X-17A-5-SPLIT-PDFS/Assets/"
    pdf_liable_folder = "Output/X-17A-5-SPLIT-PDFS/Liability & Equity/"
    
    png_asset_folder = "Output/X-17A-5-SPLIT-PNGS/Assets/"
    png_liable_folder = "Output/X-17A-5-SPLIT-PNGS/Liability & Equity/"
    
    out_folder = "Output/"

    # Amazon Textract client and Sagemaker session
    s3 = boto3.client('s3')
    session = Session()
    
    # ==============================================================================
    # ALL TEMPORARY FILE INFORMATION 
    # ==============================================================================
    # retrieving CIK-Dealers JSON file from s3 bucket
    s3.download_file(bucket, 'Temp/CIKandDealers.json', 'temp.json')
    with open('temp.json', 'r') as f: cik2brokers = json.loads(f.read())

    # remove local file after it has been created (variable is stored in memory)
    os.remove('temp.json')
    # ==============================================================================
    
    # s3 paths where asset and liability paths are stored
    asset_paths = session.list_s3_files(bucket, pdf_asset_folder)
    liable_paths = session.list_s3_files(bucket, pdf_liable_folder)
    
    # intialize list to store dataframes for asset and liability & equity
    asset_concat = [0] * len(asset_paths)
    liable_concat = [0] * len(liable_paths)
    
    # --------------------------------------------
    # Asset Unstructured Database
    # --------------------------------------------
    print('Assets Unstructured Database')
    for idx, csv in enumerate(asset_paths):
        
        # decompose csv name into corresponding terms
        fileName, filing_date, fiscal_year, cik = extra_cols(csv)
        
        # first load in both the PNG and PDF split balance sheets
        # NOTE: All these balance sheets are cleaned numerical values
        try:
            s3.download_file(bucket, csv, 'temp.csv')
            pdf_df = pd.read_csv('temp.csv')
            s3.download_file(bucket, png_asset_folder + fileName, 'temp.csv')
            png_df = pd.read_csv('temp.csv')
            os.remove('temp.csv')

            print('Working on {}-{}'.format(cik, filing_date))

            # do a special merge that combines unique line items names between PDF & PNG
            temp_df1 = special_merge(pdf_df, png_df, '0')

            # run accounting check to remove sub-totals for each respective line-item
            df, total_flag, total_amt = totals_check(temp_df1)
            
            # construct row for the unstructured data frame 
            export_df = unstructured_data(df, filing_date, fiscal_year, cik, cik2brokers)
            
            # we have that no "total asset" figure was found
            if total_flag == 2:
                export_df["Total asset check"] = "Total asset not found"
                export_df["Total asset"] = total_amt

            # we have that "total asset" was found and matches
            elif total_flag == 1:
                export_df["Total asset check"] = "Total asset found & match"
                export_df["Total asset"] = total_amt

            # we have that "total asset" was found, but did not match correctly
            elif total_flag == 0:
                export_df["Total asset check"] = "Total asset found & no match"

            # stores the reported data frame 
            asset_concat[idx] = export_df
        
        # in the event we can't download file from s3 (i.e. does not exist, we ignore the )
        except botocore.exceptions.ClientError:
            
            # assign an empty DataFrame and print out error
            asset_concat[idx] = pd.DataFrame()
            
            print('\nCLIENT-ERROR: WE COULD NOT DOWNLOAD SPLIT DATA FOR {}\n'.format(fileName))
     
    print('\n\n\n\n')
        
    # --------------------------------------------
    # Liability & Equity Unstructured Database
    # --------------------------------------------
    print('\nLiability & Equity Unstructured Database')
    for idx, csv in enumerate(liable_paths):
        
        # decompose csv name into corresponding terms
        fileName, filing_date, fiscal_year, cik = extra_cols(csv)
        
        try:
            # first load in both the PNG and PDF split balance sheets
            # NOTE: All these balance sheets are cleaned numerical values
            s3.download_file(bucket, csv, 'temp.csv')
            pdf_df = pd.read_csv('temp.csv')
            s3.download_file(bucket, png_liable_folder + fileName, 'temp.csv')
            png_df = pd.read_csv('temp.csv')
            os.remove('temp.csv')

            print('Working on {}-{}'.format(cik, filing_date))

            # do a special merge that combines unique line items names between PDF & PNG
            temp_df1 = special_merge(pdf_df, png_df, '0')

            # run accounting check to remove sub-totals for each respective line-item
            df, total_flag, total_amt = totals_check(temp_df1)
            
            # construct row for the unstructured data frame 
            export_df = unstructured_data(df, filing_date, fiscal_year, cik, cik2brokers)
            
            # we have that no "total asset" figure was found
            if total_flag == 2:
                export_df["Total liabilities & shareholder's equity check"] = "Total liabilities & shareholder's equity not found"
                export_df["Total liabilities & shareholder's equity"] = total_amt

            # we have that "total asset" was found and matches
            elif total_flag == 1:
                export_df["Total liabilities & shareholder's equity check"] = "Total liabilities & shareholder's equity found & match"
                export_df["Total liabilities & shareholder's equity"] = total_amt
        
            # we have that "total asset" was found, but did not match correctly
            elif total_flag == 0:
                export_df["Total liabilities & shareholder's equity check"] = "Total liabilities & shareholder's equity found & no match"
                
            # stores the reported data frame 
            liable_concat[idx] = export_df
        
        # in the event we can't download file from s3 (i.e. does not exist, we ignore the )
        except botocore.exceptions.ClientError:
            
            # assign an empty DataFrame and print out error
            liable_concat[idx] = pd.DataFrame()
            
            print('\nCLIENT-ERROR: WE COULD NOT DOWNLOAD SPLIT DATA FOR {}\n'.format(fileName))
    
    # --------------------------------------------
    # Database exportation
    # --------------------------------------------
    
    # writing data frame to .csv file
    asset_df = pd.concat(asset_concat)        # asset dataframe combining all rows from 
    asset_df = reorder_columns(asset_df,      # re-order columns for dataframe
                               col_preserve=['CIK', 'Name', 'Filing Date', 'Filing Year', 
                                             'Total asset check'])      
    
    filename1 = '/home/ec2-user/SageMaker/SEC_X17A5/output/unstructured_assets.csv'
    asset_df.to_csv(filename1, index=False)
    with open(filename1, 'rb') as data:
        s3.put_object(Bucket=bucket, Key=out_folder + 'unstructured_assets.csv', Body=data)
    
    
    # writing data frame to .csv file
    liable_df = pd.concat(liable_concat)     
    liable_df = reorder_columns(liable_df, 
                                col_preserve=['CIK', 'Name', 'Filing Date', 'Filing Year', 
                                              "Total liabilities & shareholder's equity check"])    
    
    filename2 = '/home/ec2-user/SageMaker/SEC_X17A5/output/unstructured_liable.csv'
    liable_df.to_csv(filename2, index=False)
    with open(filename2, 'rb') as data:
        s3.put_object(Bucket=bucket, Key=out_folder + 'unstructured_liable.csv', Body=data)
    
    print('\nWe created an unstructured asset and liability & equity')


We created an unstructured asset and liability & equity


In [ ]:
# # work on combining columns that are issued seperately
# s3 = boto3.client('s3')
# session = Session()
# bucket = "ran-s3-systemic-risk"

# # e.g. file name = 1224385-2004-03-01, 42352-2003-01-28
# s3.download_file(bucket, 'Output/X-17A-5-SPLIT-PDFS/Assets/42352-2011-03-01.csv', 'temp.csv')
# pdf_df = pd.read_csv('temp.csv')
# s3.download_file(bucket, 'Output/X-17A-5-SPLIT-PNGS/Assets/42352-2011-03-01.csv', 'temp.csv')
# png_df = pd.read_csv('temp.csv')
# os.remove('temp.csv')

# # do a special merge that combines unique line items names between PDF & PNG
# temp_df1 = special_merge(pdf_df, png_df, '0')

# # run accounting check to remove sub-totals for each respective line-item
# df, total_flag, total_amt = totals_check(temp_df1)


In [ ]:
# temp_df1

In [ ]:
# totals_check(temp_df1)